### Setup

In [2]:
import os, sys
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='cobra')

from cobra import io
from scripts.helpers.model import rxn_in_model, add_single_gene_reaction_pair
from scripts.opt._fba import flux_balance_analysis
from scripts.opt._fva import run_flux_variability_analysis

### Import Models

In [3]:
# Load models

# Load wildtype from manual directory (adjust path for notebooks directory)
# Use io.read_sbml_model for local files instead of io.load_model
wildtype = io.read_sbml_model('../data/manual/xmls/MNL_iCre1355_GAPFILL.xml')

models = {
    "Wildtype": wildtype
}

# Get altered reactions of wildtype - using correct directory name
altered_dir = '../data/altered/xmls/MNL_iCre1355_GAPFILL/'

for root, dirs, files in os.walk(altered_dir):
    for file in files:
        if file.endswith('.xml'):
            # Get file name and remove .xml extension
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading model: {model_name} from {full_path}")
            models[model_name] = io.read_sbml_model(full_path)

# Print out loaded models
print(f"\nLoaded {len(models)} models:")
for name in models.keys():
    print(f"  - {name}: {models[name].name}")

No objective coefficients in model. Unclear what should be optimized


Loading model: SQE+MVA from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQE+MVA.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQE from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQE.xml


No objective coefficients in model. Unclear what should be optimized


...

Loading model: SQS+SQE from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQS+SQE.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQS from ../data/altered/xmls/MNL_iCre1355_GAPFILL/SQS.xml


No objective coefficients in model. Unclear what should be optimized



Loaded 7 models:
  - Wildtype: None
  - SQE+MVA: MNL_iCre1355_GAPFILL_SQE+MVA
  - SQE: MNL_iCre1355_GAPFILL_SQE
  - SQS+MVA: MNL_iCre1355_GAPFILL_SQS+MVA
  - SQS+SQE+MVA: MNL_iCre1355_GAPFILL_SQS+SQE+MVA
  - SQS+SQE: MNL_iCre1355_GAPFILL_SQS+SQE
  - SQS: MNL_iCre1355_GAPFILL_SQS


### IDs and Vmax

In [4]:
from enum import Enum

class MetID(Enum):
    # Pyruvate Compounds
    IPP = "ipdp_h"
    DMAPP = "dmpp_h"
    # Sterol Compounds
    ERG = "FILL_05"
    EPI = "FILL_02"
    METLOP = "FILL_01"
    FEC = "mfecostrl_c"
    EPOXY = "Ssq23epx_c"
    SQL = "sql_c"
    # MVA Compounds
    MVA1 = "C00356"
    MVA2 = "C00418"
    MVA3 = "C01107"
    MVA4 = "C01143"
    MVA5 = "C00009"

class RxnID(Enum):
    # Biomass
    BIOMASS = "Biomass_Chlamy_auto"
    # Exchange
    EXCHERG = "ERGOSTEROLEXCH" # This will be added to models first
    # Ergosterol Synthesis
    SS = 'SS'
    SS2 = 'ALT_SQS2'
    SQE = 'SMO'
    SQE2 = 'ALT_SQE2'
    CAS = 'CAS'
    ERG = 'RXNFILL_05'
    # MVA Pathway
    MVAS = 'ALT_MVAS'
    MVAE = 'ALT_MVAE'
    MVAD = 'ALT_MVAD'
    PMK = 'ALT_PMK'
    MVK = 'ALT_MVK'
    IDLI = 'ALT_IDLI'
    # Pyruvate Metabolism

# Vmax = {
#     key: 0.005 for key in RxnID.__members__.keys()
# }

### Preprocessing Models

In [5]:
for name in models.keys():
    model = models[name]

    # Add export reaction
    try:
        add_single_gene_reaction_pair(
            model=model, 
            gene_id="EXCHERG_GENE",
            reaction_id=RxnID.EXCHERG.value,
            reaction_name="Ergosterol exchange (assumption)", 
            reaction_subsystem="Exchange", 
            metabolites=[(-1, MetID.ERG.value)],
            reversible=True
        )
    except:
        pass

    # # Add Vmax information for SQS, SQE, and MVA stuff
    # for k, v in Vmax.items():
    #     if rxn_in_model(model, k):
    #         model.reactions.get_by_id(k).lower_bound = v

### Flux Balance Analysis

In [6]:
# Run flux-balance analysis on each model and store results
fba = {}

for name, model in models.items():
    print(f"\nRunning FBA for model: {name}")
    fba[name] = flux_balance_analysis(model, objectives=[RxnID.BIOMASS.value, RxnID.EXCHERG.value], loopless=False, is_pfba=True)
    print(f"\tObjective value: {fba[name].fluxes[RxnID.EXCHERG.value]}")


Running FBA for model: Wildtype
	Objective value: 7.090681340665354e-15

Running FBA for model: SQE+MVA
	Objective value: 7.090681340665354e-15

Running FBA for model: SQE+MVA
	Objective value: 0.06896551724138

Running FBA for model: SQE
	Objective value: 0.06896551724138

Running FBA for model: SQE
	Objective value: 0.06896551724138106

Running FBA for model: SQS+MVA
	Objective value: 0.06896551724138106

Running FBA for model: SQS+MVA
	Objective value: -1.170352019085644e-14

Running FBA for model: SQS+SQE+MVA
	Objective value: -1.170352019085644e-14

Running FBA for model: SQS+SQE+MVA
	Objective value: 0.06896551724138013

Running FBA for model: SQS+SQE
	Objective value: 0.06896551724138013

Running FBA for model: SQS+SQE
	Objective value: 0.06896551724137949

Running FBA for model: SQS
	Objective value: 0.06896551724137949

Running FBA for model: SQS
	Objective value: 1.7190529524578766e-14
	Objective value: 1.7190529524578766e-14


### Flux Variability Analysis

In [7]:
fva = {}

target_rxns = [
    RxnID.BIOMASS.value,
    RxnID.EXCHERG.value,
    RxnID.ERG.value,
    RxnID.SS.value,
    RxnID.SS2.value,
    RxnID.SQE.value,
    RxnID.SQE2.value,
    RxnID.CAS.value,
    RxnID.MVAD.value,
    RxnID.MVAE.value,
    RxnID.MVAS.value,
    RxnID.PMK.value,
    RxnID.MVK.value,
    RxnID.IDLI.value
]

for name, model in models.items():
    print(f"\nRunning FVA for model: {name}")
    fva[name] = run_flux_variability_analysis(
        model,
        loopless=True,
        pfba_factor=None,
        objectives=[RxnID.EXCHERG.value],
        reactions=[rxn for rxn in target_rxns if rxn_in_model(model, rxn)]
    )
    print(fva[name])


Running FVA for model: Wildtype
                     minimum       maximum
Biomass_Chlamy_auto     0.00  3.854623e-17
ERGOSTEROLEXCH          0.04  4.000000e-02
RXNFILL_05              0.04  4.000000e-02
SS                      0.04  4.000000e-02
SMO                     0.04  4.000000e-02
CAS                     0.04  4.000000e-02

Running FVA for model: SQE+MVA
                     minimum       maximum
Biomass_Chlamy_auto     0.00  3.854623e-17
ERGOSTEROLEXCH          0.04  4.000000e-02
RXNFILL_05              0.04  4.000000e-02
SS                      0.04  4.000000e-02
SMO                     0.04  4.000000e-02
CAS                     0.04  4.000000e-02

Running FVA for model: SQE+MVA
                          minimum       maximum
Biomass_Chlamy_auto  0.000000e+00  1.243184e-15
ERGOSTEROLEXCH       6.896552e-02  6.896552e-02
RXNFILL_05           6.896552e-02  6.896552e-02
SS                   6.896552e-02  6.896552e-02
SMO                 -1.530359e-16  4.647404e-15
ALT_SQE2     

### Results

In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Define target reactions for heatmap
target_reactions = [
    RxnID.EXCHERG.value,
    RxnID.BIOMASS.value,
    RxnID.ERG.value
]

# Create data for heatmap
heatmap_data = []
model_names = []
flux_matrix = []

for model_name, fba_result in fba.items():
    model_names.append(model_name)
    flux_row = []
    
    for rxn_id in target_reactions:
        # Get flux value, default to 0 if reaction not found
        if rxn_id in fba_result.fluxes.index:
            flux_value = fba_result.fluxes[rxn_id]
        else:
            flux_value = 0.0
        flux_row.append(flux_value)
    
    flux_matrix.append(flux_row)

# Create DataFrame for easier handling
df_heatmap = pd.DataFrame(
    flux_matrix, 
    index=model_names, 
    columns=target_reactions
)

print("FBA Flux Heatmap Data:")
print(df_heatmap)

# Create heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=df_heatmap.values,
    x=target_reactions,
    y=model_names,
    colorscale='RdYlBu_r',  # Red-Yellow-Blue reversed (red=high, blue=low)
    text=df_heatmap.values.round(4),  # Show values on cells
    texttemplate="%{text}",
    textfont={"size": 10},
    colorbar=dict(title="Flux (mmol/gDW/h)")
))

fig.update_layout(
    title="FBA Flux Analysis Heatmap",
    xaxis_title="Reactions",
    yaxis_title="Models",
    width=800,
    height=600,
    font=dict(size=12)
)

fig.show()

FBA Flux Heatmap Data:
             ERGOSTEROLEXCH  Biomass_Chlamy_auto    RXNFILL_05
Wildtype       7.090681e-15         5.157516e-02  7.095648e-15
SQE+MVA        6.896552e-02         0.000000e+00  6.896552e-02
SQE            6.896552e-02        -1.842254e-15  6.896552e-02
SQS+MVA       -1.170352e-14         5.157514e-02 -1.169592e-14
SQS+SQE+MVA    6.896552e-02         0.000000e+00  6.896552e-02
SQS+SQE        6.896552e-02         0.000000e+00  6.896552e-02
SQS            1.719053e-14         5.157514e-02  1.719053e-14


In [14]:
# Export results
import os

# Create results directory if it doesn't exist
results_dir = '../results/bench/heatmap'
os.makedirs(results_dir, exist_ok=True)

# Save the heatmap as PNG
output_path = os.path.join(results_dir, 'fba_flux_heatmap.png')
fig.write_image(output_path, width=800, height=600, scale=2)
print(f"Heatmap saved to: {output_path}")

# Also save the DataFrame as CSV for reference
csv_path = os.path.join(results_dir, 'fba_flux_data.csv')
df_heatmap.to_csv(csv_path)
print(f"Flux data saved to: {csv_path}")

# Save the heatmap as HTML for interactive viewing
html_path = os.path.join(results_dir, 'fba_flux_heatmap.html')
fig.write_html(html_path)
print(f"Interactive heatmap saved to: {html_path}")

# Save the heatmap as .png for convenience
png_path = os.path.join(results_dir, 'fba_flux_heatmap.png')
fig.write_image(
    png_path,
    width=1200,
    height=800,
    scale=2,
    format="png"
)
print(f"Image (PNG) heatmap saved to: {png_path}")

Heatmap saved to: ../results/bench/heatmap\fba_flux_heatmap.png
Flux data saved to: ../results/bench/heatmap\fba_flux_data.csv
Interactive heatmap saved to: ../results/bench/heatmap\fba_flux_heatmap.html
Image (PNG) heatmap saved to: ../results/bench/heatmap\fba_flux_heatmap.png
Image (PNG) heatmap saved to: ../results/bench/heatmap\fba_flux_heatmap.png
